In [6]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

from collections import Counter
# Lemmatize with POS Tag
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [2]:
D = np.load('tf_idf_vector_matrix.npy') # load

KeyboardInterrupt: 

In [ ]:
D1 = np.load('count_vector_matrix.npy')

In [ ]:
D.shape

In [4]:
D

array([[0.03311033, 0.20788222, 0.19452914, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.12945549, 0.12945549,
        0.        ],
       [0.0148572 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04315183]])

In [3]:
D1.shape

(9025, 19931)

In [4]:
D1

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Import Data

In [9]:
md = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')

In [10]:
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [11]:
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9020,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"['Comedy', 'Horror', 'Science Fiction']",http://www.syfy.com/sharknado4,390989,tt4831420,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,...,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What happens in Vegas, stays in Vegas. Unless ...",Sharknado 4: The 4th Awakens,False,4.3,88.0,The new installment of the Sharknado franchise...,2016
9021,False,NaN,8000000,['Drama'],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,A man must cope with the loss of his wife and ...,2001
9022,False,NaN,1000000,"['Thriller', 'Romance']",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"Rustom Pavri, an honourable officer of the Ind...",2016
9023,False,NaN,15050000,"['Adventure', 'Drama', 'History', 'Romance']",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,155.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"Village lad Sarman is drawn to big, bad Mohenj...",2016


In [14]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(
        1, 2), min_df=0, stop_words='english')

md['tf-idf-ver1'] = md['description'] + md['title']
md['tf-idf-ver1'] = md['tf-idf-ver1'].fillna('')

In [ ]:
tfidf_matrix = tf.fit_transform(md['tf-idf-ver1'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
train

,userId,id,rating,movieId,timestamp,movieIndex
0,0,1598.0,2.0,1343,1260759131,1083
1,0,783.0,2.0,1293,1260759148,1043
2,0,97.0,4.0,2105,1260759139,1661
3,0,11778.0,2.0,1263,1260759151,1013
4,0,847.0,2.0,2193,1260759198,1739
...,...,...,...,...,...,...
79843,670,8879.0,4.0,2401,1065149286,1907
79844,670,137.0,4.0,1265,1063503895,1015
79845,670,11663.0,4.0,3060,1063503947,2453
79846,670,117.0,3.5,2194,1064891593,1740


In [9]:
test

,userId,id,rating,movieId,timestamp,movieIndex
0,0,11072.0,3.0,3671,1260759117,2920
1,0,152.0,2.5,1371,1260759135,1107
2,0,8393.0,3.0,2150,1260759194,1704
3,0,11216.0,4.0,1172,1260759205,927
4,0,3179.0,1.0,1405,1260759203,1136
...,...,...,...,...,...,...
19957,670,8346.0,3.0,5299,1065112004,4043
19958,670,824.0,3.5,4308,1065111985,3413
19959,670,51927.0,4.0,6269,1065149201,4538
19960,670,745.0,4.0,2762,1064891036,2208


In [10]:
cosine_sim = linear_kernel(D, D)

In [3]:
cosine_sim1 = linear_kernel(D1, D1)

In [11]:
cosine_sim.shape

(9025, 9025)

In [4]:
cosine_sim1.shape

(9025, 9025)

In [5]:
cosine_sim1

array([[24.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1., 21.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0., 17., ...,  1.,  1.,  0.],
       ...,
       [ 0.,  0.,  1., ..., 15.,  2.,  0.],
       [ 0.,  1.,  1., ...,  2., 17.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 16.]])

In [16]:
np.save('test_cosine_sim', cosine_sim)

In [19]:
cosine_sim = np.load('test_cosine_sim.npy')

In [ ]:
np.save('cosine_sim_ver2', cosine_sim1)

In [30]:
cosine_sim1 = np.load('cosine_sim_ver2.npy')

MemoryError: Unable to allocate array with shape (81450625,) and data type float64